In [1]:
import tensorflow as tf

2024-10-12 09:29:23.337756: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-12 09:29:23.340708: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-12 09:29:23.351101: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-12 09:29:23.367865: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-12 09:29:23.372181: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-12 09:29:23.384361: I tensorflow/core/platform/cpu_feature_gu

In [2]:
# create eff_imagenet model and save in saved model format
model = tf.keras.applications.EfficientNetB0(weights= "imagenet", include_top = True)
for layer in model.layers:
    layer.trainable = False
model.export("eff-imagenet/1")

INFO:tensorflow:Assets written to: eff-imagenet/1/assets


INFO:tensorflow:Assets written to: eff-imagenet/1/assets


Saved artifact at 'eff-imagenet/1'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 1000), dtype=tf.float32, name=None)
Captures:
  134894970086864: TensorSpec(shape=(1, 1, 1, 3), dtype=tf.float32, name=None)
  134894970086096: TensorSpec(shape=(1, 1, 1, 3), dtype=tf.float32, name=None)
  134894970087248: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134894970086288: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134894970083024: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134894970087056: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134894970086672: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134894970089168: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134894970089552: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134894970085904: TensorSpec(shape=(), dtype=tf.resource, name=Non

In [14]:
# load model and save in saved_model_format
model_path = "skin_cancer_model_effb0.keras"
save_path = "/model/skin_cancer_model/1"
model = tf.keras.models.load_model(model_path)
model.export(save_path)

INFO:tensorflow:Assets written to: test/1/assets


INFO:tensorflow:Assets written to: test/1/assets


Saved artifact at 'test/1'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input_layer')
Output Type:
  TensorSpec(shape=(None, 7), dtype=tf.float32, name=None)
Captures:
  139378773891344: TensorSpec(shape=(1, 1, 1, 3), dtype=tf.float32, name=None)
  139378773891728: TensorSpec(shape=(1, 1, 1, 3), dtype=tf.float32, name=None)
  139376700206416: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139376700205840: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139376700209296: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139376700207184: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139376700206800: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139376700204496: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139376700203920: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139376700204688: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1393767

## Add base64 serving

In [55]:
model = tf.saved_model.load("eff-imagenet/1")
signature_default = model.signatures["serving_default"]

@tf.function(input_signature=[tf.TensorSpec(name="image_bytes_string", shape=None, dtype=tf.string)])
def predict_bytes_image(image):
    """
    Predict using encoded image bytes.
    :param image: png, jpeg, bmp, gif encoded image bytes.
    :return: prediction result.
    """
    image = tf.reshape(image, [])
    image = tf.io.decode_image(image, channels=3)
    image = tf.cast(image, dtype=tf.float32)
    image = tf.expand_dims(image, 0)

    return signature_default(image)

signatures = dict()
signatures["serving_default"] = signature_default
signatures["serving_bytes"] = predict_bytes_image

tf.saved_model.save(
    model,
    export_dir="eff-imagenet/2",
    signatures=signatures
)

INFO:tensorflow:Assets written to: eff-imagenet/2/assets


INFO:tensorflow:Assets written to: eff-imagenet/2/assets


In [24]:
model = tf.saved_model.load("eff-imagenet/1")
# This is the current signature, that only accepts image tensors as input
signature = model.signatures["serving_default"]

@tf.function()
def my_predict(image_b64):
    # Model doesn't support batch!!
    img_dec = tf.image.decode_image(image_b64[0], channels=3)
    img_tensor = tf.image.convert_image_dtype(img_dec, tf.float32)[tf.newaxis, ...]
    prediction = signature(img_tensor)
    return prediction


# Create new signature, to read b64 images
new_signature = my_predict.get_concrete_function(
    image_b64=tf.TensorSpec([None], dtype=tf.string, name="image_b64")
)

tf.saved_model.save(
    model,
    export_dir="eff-imagenet/2",
    signatures=new_signature
)

INFO:tensorflow:Assets written to: eff-imagenet/2/assets


INFO:tensorflow:Assets written to: eff-imagenet/2/assets
